# Lecture 13: Agentic Memory

### Understanding Chatbots and effective token usage (trim, filtering, summary) in pure python and into to agentic memory

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

# Corrected way to retrieve the API key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's set in the .env file.")

os.environ["GOOGLE_API_KEY"] = api_key

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

### Vanilla Chatbot

In [2]:
# Set the system message
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="What is the meaning of life?"),
    AIMessage(content="The meaning of life is to find happiness."),
]

while True:
    user_input = input("Enter your message: ")
    if user_input.lower() == "exit":
        break
    messages.append(HumanMessage(content=user_input))
    print(messages)
    response = llm.invoke(messages)
    messages.append(AIMessage(content=response.content))
    print(response.content)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the meaning of life?', additional_kwargs={}, response_metadata={}), AIMessage(content='The meaning of life is to find happiness.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Indian Food', additional_kwargs={}, response_metadata={})]
Indian food is incredibly diverse, varying greatly by region, religion, and even family traditions.  It's not possible to give a single description, but here are some key characteristics and aspects:

* **Regional Variations:**  North Indian cuisine differs significantly from South Indian, West Indian, and East Indian food.  For example, North India is known for its tandoori dishes, butter chicken, and naan bread, while South India features dosas, idlis, sambar, and a greater use of coconut.  Each region has its own unique spices, cooking techniques, and staple ingredients.

* **Spices:**  Indian cooking is

## Problems

### 1. Long conversation history results in high token usage and slow response times and cost

## Filtering

In [3]:
# Same code just top 3 messages with system message always included

#Set the system message
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage


messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Explain the concept of AI?"),
    AIMessage(content="AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.")
]

while True:
    user_input = input("Enter your message: ")
    if user_input.lower() == "exit":
        break
    messages.append(HumanMessage(content=user_input))
    print("Full messages", messages)

    try:
        # Always include system message + last 2 messages
        filtered_messages = [messages[0]] + messages[-2:]
        print("Filtered messages", filtered_messages)
        response = llm.invoke(filtered_messages)
        messages.append(AIMessage(content=response.content))
        print(response.content)
    except Exception as e:
        print("Error:", e)
        response = llm.invoke(messages) # Fallback to full history
        messages.append(AIMessage(content=response.content))
        print(response.content)


Full messages [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain the concept of AI?', additional_kwargs={}, response_metadata={}), AIMessage(content='AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Indian food', additional_kwargs={}, response_metadata={})]
Filtered messages [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), AIMessage(content='AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Indian food', additional_kw

## Trimming

### Same code just trim last n words or tokens

Set the system message


In [4]:
# Same code trimmed to last 100 words of conversation with system message

#Set the system message
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage


def get_last_n_words(messages, n=100):
    """Get the last n words from a list of messages."""
    total_words = 0
    trimmed_messages = []
    # Iterate over the messages in reverse order (excluding system message)
    for msg in reversed(messages[1:]):
        words = msg.content.split() # split the content into words
        if total_words + len(words) > n:
            words_to_take = n - total_words
            if words_to_take > 0:
                trimmed_content = ' '.join(words[-words_to_take])
                if isinstance(msg, HumanMessage):
                    trimmed_messages.insert(0, HumanMessage(content=trimmed_content))
                elif isinstance(msg, AIMessage):
                    trimmed_messages.insert(0, AIMessage(content=trimmed_content))
                total_words += words_to_take
            break
        else:
            trimmed_messages.insert(0, msg)
            total_words += len(words)
        if total_words >= n:
            break
    return trimmed_messages

messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Explain the concept of AI?"),
    AIMessage(content="AI is a technology that allows computers to learn and improve from experience, without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions.")
]

while True:
    user_input = input("Enter your message: ")
    if user_input.lower() == "exit":
        break
    messages.append(HumanMessage(content=user_input))
    print("Full messages: ", messages)

    try:
        # Trim the conversation to the last 100 words
        trimmed_conversation = get_last_n_words(messages, 100)
        # Always include system message
        filtered_messages = [messages[0]] + trimmed_conversation
        print("Trimmed messages: ", filtered_messages)
        response = llm.invoke(filtered_messages)
        messages.append(AIMessage(content=response.content))
        print(response.content)
    except Exception as e:
        print("Error:", e)
        response = llm.invoke(messages)  # Fallback to full history
        messages.append(AIMessage(content=response.content))
        print(response.content)


Homework: Take a case in every 10 message u will summarize the conversation and send current input + summary rather than entire text

Note: This is simple implementation often u would need to implement at a session and level and other levels of memory